In [2]:
import gmsh
import numpy as np

In [3]:
# import sys

# # Initialize Gmsh
# gmsh.initialize()

# try:
#     # Create a new model named "circle"
#     gmsh.model.add("circle")

#     # Define parameters
#     radius = 1.0       # Radius of the circle
#     mesh_size = 0.5    # Approximate mesh element size

#     # Use the OpenCASCADE (OCC) geometry kernel
#     # Add a circle centered at (0, 0, 0) with radius `radius`
#     # The resulting curve has tag 1
#     gmsh.model.occ.addCircle(0, 0, 0, radius, 1)

#     # Create a curve loop from the circle (curve tag 1)
#     # The resulting curve loop has tag 1
#     gmsh.model.occ.addCurveLoop([1], 1)

#     # Create a plane surface from the curve loop (loop tag 1)
#     # The resulting surface has tag 1
#     gmsh.model.occ.addPlaneSurface([1], 1)

#     # Synchronize the OCC geometry with the Gmsh model
#     gmsh.model.occ.synchronize()

#     # Set the global mesh size
#     # This will be used as the target size for mesh elements
#     gmsh.option.setNumber("Mesh.CharacteristicLengthMax", mesh_size)

#     # Generate the 2D mesh (dim=2)
#     gmsh.model.mesh.generate(2)

#     # Save the mesh to a file
#     output_file = "circle.msh"
#     gmsh.write(output_file)
#     print(f"Mesh successfully generated and saved to '{output_file}'")

#     # Launch the Gmsh GUI to visualize the result
#     # This will open a new window unless '-nopopup' is passed as an argument
#     if '-nopopup' not in sys.argv:
#         print("Opening Gmsh GUI to view mesh... (Close GUI to exit script)")
#         gmsh.fltk.run()

# except Exception as e:
#     print(f"An error occurred: {e}")

# finally:
#     # Finalize Gmsh
#     gmsh.finalize()

In [ ]:
# >> K=size(t,1);  % Number of elements
# >> Nv=size(p,1); % Number of vertices in mesh
# >> Nfaces=size(t,2); % Number of faces/element
# >> VX = p(:,1); % Vertice x-coordinates
# >> VY = p(:,2); % Vertice y-coordinates
# >> EToV = t; % Element to Vertice table



In [22]:
gmsh.initialize()

# Generate a circle
radius = 1.0
origin = (0, 0, 0)

# Mesh size
# Cannot directly set number of nodes, so tune the parameter below to get roughly the number of meshes you want
mesh_size = 1.0

gmsh.model.add("circle")
tag_circle = gmsh.model.occ.addCircle(*origin, radius)
tag_loop = gmsh.model.occ.addCurveLoop([tag_circle])
tag_surface = gmsh.model.occ.addPlaneSurface([tag_loop])

# Sync to model
gmsh.model.occ.synchronize()

gmsh.option.setNumber("Mesh.CharacteristicLengthMax", mesh_size)
# Generate mesh
gmsh.model.mesh.generate(dim=2)

# Now we validate our mesh
element_types, element_tags, node_tags_element = gmsh.model.mesh.getElements(dim=2, tag=tag_surface)
assert len(element_types) == 1, "Unexpected number of element types"
element_type = element_types[0]
num_elements = len(element_tags[0])

name, dim, order, num_nodes_per_element, local_node_coord, num_primary_nodes = gmsh.model.mesh.getElementProperties(element_type)
assert dim == 2, "Element dimension mismatch"
assert num_nodes_per_element == 3, "We want triangular mesh, something went wrong"

# Construct EtoV
# nodeTags[0] contains the flattened list of node tags for all elements
# We can reshape this using numpy for easier viewing
# The list is [e1_n1, e1_n2, e1_n3, e2_n1, e2_n2, e2_n3, ...]
EtoV = np.array(node_tags_element).reshape(num_elements, num_nodes_per_element)

node_tags, node_coords, _ = gmsh.model.mesh.getNodes(dim=2, tag=tag_surface)
# node_coords is flattened: [x1, y1, z1, x2, y2, z2, ...]
node_coords = np.array(node_coords).reshape(-1, 3)
V_x = node_coords[:, 0]
V_y = node_coords[:, 1]

num_nodes = len(node_tags)

# Planar graph, so _obviouslyly_, number of vertices is simply:
num_faces = 2 + num_elements - num_nodes


# Get node coordinates

# >> K=size(t,1);  % Number of elements
# >> Nv=size(p,1); % Number of vertices in mesh
# >> Nfaces=size(t,2); % Number of faces/element
# >> VX = p(:,1); % Vertice x-coordinates
# >> VY = p(:,2); % Vertice y-coordinates
# >> EToV = t; % Element to Vertice table

# gmsh.fltk.run()
gmsh.finalize()

print(f"K:{num_elements=}")
print(f"Nv:{num_nodes=}")
print(f"Nfaces:{num_faces=}")
print(f"VX:{V_x=}")
print(f"VY:{V_y=}")
print(f"EToV:{EtoV=}")


K:num_elements=117
Nv:num_nodes=48
Nfaces:num_faces=71
VX:V_x=array([ 0.74780779, -0.71707393, -0.15357641,  0.08628135, -0.64494857,
        0.57679749,  0.44406617, -0.35030003, -0.3821794 , -0.23684933,
        0.00614936, -0.08338813,  0.17445398,  0.08157309, -0.18640673,
       -0.01962153,  0.2441725 , -0.2740722 ,  0.13858496, -0.46437844,
       -0.3461251 ,  0.33697241,  0.5418855 , -0.14124029,  0.44607512,
        0.08634328,  0.42205057,  0.30253067,  0.69580244, -0.36116705,
       -0.54694383, -0.5117279 , -0.73489993, -0.56394708,  0.69902661,
       -0.12942782,  0.62522809,  0.23912241, -0.70010018,  0.55049808,
        0.29022389,  0.7896826 , -0.56198583, -0.44021579,  0.29071013,
        0.48306431,  0.03893205, -0.77902497])
VY:V_y=array([-0.10278381,  0.19409253, -0.73905026,  0.77956322, -0.39220191,
        0.51319026, -0.62147959,  0.6663462 , -0.68531333, -0.48827635,
       -0.51984091, -0.27878836, -0.30761793, -0.06333455, -0.02802458,
        0.18730831, 

In [20]:
print(node_coords)

[ 0.74780779 -0.10278381  0.         -0.71707393  0.19409253  0.
 -0.15357641 -0.73905026  0.          0.08628135  0.77956322  0.
 -0.64494857 -0.39220191  0.          0.57679749  0.51319026  0.
  0.44406617 -0.62147959  0.         -0.35030003  0.6663462   0.
 -0.3821794  -0.68531333  0.         -0.23684933 -0.48827635  0.
  0.00614936 -0.51984091  0.         -0.08338813 -0.27878836  0.
  0.17445398 -0.30761793  0.          0.08157309 -0.06333455  0.
 -0.18640673 -0.02802458  0.         -0.01962153  0.18730831  0.
  0.2441725   0.14782961  0.         -0.2740722   0.22186779  0.
  0.13858496  0.38172412  0.         -0.46437844  0.01714834  0.
 -0.3461251  -0.24122888  0.          0.33697241 -0.10483228  0.
  0.5418855   0.06812158  0.         -0.14124029  0.46536723  0.
  0.44607512 -0.36720476  0.          0.08634328 -0.7551626   0.
  0.42205057  0.33048608  0.          0.30253067  0.59294146  0.
  0.69580244  0.29833906  0.         -0.36116705  0.43984591  0.
 -0.54694383  0.54146951 